In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from pathlib import Path
import datetime

In [ ]:
!where

In [ ]:
today = datetime.datetime.now().strftime("%Y-%m-%d")
csv_path = Path(f"binance_trades_{today}.csv") # Mise en place d'un nouveau fichier charque jours si il existe

In [ ]:
if csv_path.exists():
    df = pd.read_csv(csv_path)
    print(f"Le fichier {csv_path} à été chargé") # Condition pour entrainer le modele

In [ ]:
# Nettoyage et conversion
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df.dropna(inplace=True)

In [ ]:
# Créer la variable cible : le prix du prochain trade
df['next_price'] = df['price'].shift(-1)
df.dropna(inplace=True)

In [ ]:
# Définir les features et la target
X = df[['price', 'quantity']]
y = df['next_price']

In [ ]:
# Séparer les données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entraîner un modèle de régression
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Évaluer le modèle
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [ ]:
print("MAE:", mae)
print("RMSE:", rmse)
print("R2 Score:", r2)

In [ ]:
# Sauvegarder le modèle pour l'utiliser plus tard dans une API
joblib.dump(model, "model.pkl")
print("✅ Modèle enregistré sous model.pkl")

In [ ]:
else :
    print(f"Aucun fichier trouvé le {"today"}") # Si aucun fichier ce jour